In [2]:
from tinydb import TinyDB, Query, middlewares, table, where
from tinydb.storages import JSONStorage
import json
import ast
import numpy as np
import scipy.optimize
import pylab as plt
import pandas as pd

In [3]:
db = TinyDB('db.json')
User = Query()

In [4]:
#db.remove(User.city.climate.climateMonth == [])

In [5]:
def test_func(x, a, b, c):
    return a * np.sin(np.pi*b*x/6) + c

In [23]:
idx = [6,7,8,9,10,11,0,1,2,3,4,5]
months = np.arange(1,13)

def parametergeneration(x, months):
    locale = db.get(doc_id=x)
    if locale != None:
        wthrsource = (locale['city']['climate']['climateMonth'])
        #evaluating for maximum temperature, minimum temperature, and rainfall
        try:
            maxtemptest = np.array([float(wthrsource[i]['maxTemp']) for i in range(len(wthrsource))])
        except:
            maxtemptest = np.array([None])
        try:
            mintemptest = np.array([float(wthrsource[i]['minTemp']) for i in range(len(wthrsource))])
        except:
            mintemptest = np.array([None])
        try:
            rainfalltest = np.array([float(wthrsource[i]['rainfall']) for i in range(len(wthrsource))])
        except:
            rainfalltest = np.array([None])
        #corrects for southern hemisphere locations
        if float(locale['city']['cityLatitude']) < 0:
            try:
                maxtemptest = maxtemptest[idx]
            except:
                pass
            try:
                mintemptest = mintemptest[idx]
            except:
                pass
            try:
                rainfalltest = rainfalltest[idx]
            except:
                pass
        return maxtemptest, mintemptest, rainfalltest
    else:
        return None   

In [24]:
def dataframecreator(database):
    headers = ['City', 'Country', 'Latitude', 'Longitude',
            'MaxTemp1', 'MaxTemp2', 'MaxTemp3', 'MaxTemp4', 'MaxTemp5', 'MaxTemp6'
            'MaxTemp7', 'MaxTemp8', 'MaxTemp9', 'MaxTemp10', 'MaxTemp11', 'MaxTemp12'
            'MinTemp1', 'MinTemp2', 'MinTemp3', 'MinTemp4', 'MinTemp5', 'MinTemp6',
            'MinTemp7', 'MinTemp8', 'MinTemp9', 'MinTemp10', 'MinTemp11', 'MinTemp12',
            'Rainfall1', 'Rainfall2', 'Rainfall3','Rainfall4', 'Rainfall5', 'Rainfall6',
            'Rainfall7', 'Rainfall8', 'Rainfall9','Rainfall10', 'Rainfall11', 'Rainfall12']
    skeleton = [headers]
    k = skeleton
    for i in range(3500):
        locale = database.get(doc_id=i)
        if locale != None:
            r = []
            cityn = locale['city']['cityName']
            countryn = locale['city']['member']['memName']
            lat = float(locale['city']['cityLatitude'])
            longi = float(locale['city']['cityLongitude'])
            r = r + [cityn] + [countryn] + [lat] + [longi]
            parameters = parametergeneration(i, months)
            tick = 0
            for j in parameters:
                if j.any() != None:
                    r = r + list(parameters[tick])
                else:
                    r = r + [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
                tick += 1
            k = k + [r]
    return k
q = dataframecreator(db)
df = pd.DataFrame(dataframecreator(db))
df.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,City,Country,Latitude,Longitude,MaxTemp1,MaxTemp2,MaxTemp3,MaxTemp4,MaxTemp5,MaxTemp6MaxTemp7,...,Rainfall5,Rainfall6,Rainfall7,Rainfall8,Rainfall9,Rainfall10,Rainfall11,Rainfall12,NaN,NaN
1,Kabul,Afghanistan,34.55,69.22,4.5,5.5,12.5,19.2,24.4,30.2,...,67.9,71.9,23.4,1.0,6.2,1.6,1.7,3.7,18.6,21.6
2,Algiers,Algeria,36.7,3.22,16.7,17.4,19.2,20.9,23.9,28.2,...,55.0,58.4,41.9,8.5,4.5,8.2,28.3,58.8,89.6,91.0
3,Annaba,Algeria,36.9,7.76,16.3,16.8,18.6,20.5,23.7,27.5,...,61.2,64.1,38.3,14.0,3.1,8.2,37.5,64.8,98.4,110.8
4,Bechar,Algeria,31.61,-2.22,15.9,18.8,22.4,26.2,30.6,36.5,...,8.8,7.5,5.3,5.0,0.7,2.0,7.4,10.6,13.6,8.2


In [13]:
"""def dataframecreator(database):
    headers = ['City', 'Country', 'Latitude', 'Longitude',
            'MaxTempP1', 'MaxTempP2', 'MaxTempP3',
            'MinTempP1', 'MinTempP2', 'MinTempP3',
            'RainfallP1', 'RainfallP2', 'RainfallP3']
    skeleton = [headers]
    k = skeleton
    for i in range(3500):
        locale = database.get(doc_id=i)
        if locale != None:
            r = []
            cityn = locale['city']['cityName']
            countryn = locale['city']['member']['memName']
            lat = float(locale['city']['cityLatitude'])
            longi = float(locale['city']['cityLongitude'])
            r = r + [cityn] + [countryn] + [lat] + [longi]
            parameters = parametergeneration(i, months)
            tick = 0
            for j in parameters:
                if j.any() != None:
                    r = r + list(parameters[tick])
                else:
                    r = r + [np.nan, np.nan, np.nan]
                tick += 1
            k = k + [r]
    return k
q = dataframecreator(db)
df = pd.DataFrame(dataframecreator(db))
df.head()"""

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,City,Country,Latitude,Longitude,MaxTempP1,MaxTempP2,MaxTempP3,MinTempP1,MinTempP2,MinTempP3,RainfallP1,RainfallP2,RainfallP3
1,Kabul,Afghanistan,34.55,69.22,12.819623,-1.219531,21.382738,-29.805647,-0.438178,-16.88267,33.12913,1.046551,25.393398
2,Algiers,Algeria,36.7,3.22,-20.350724,-0.414267,8.750673,-18.965601,-0.407285,-2.049861,13.6544,10.088598,49.714661
3,Annaba,Algeria,36.9,7.76,-20.061667,-0.411448,8.46419,-17.566003,-0.40689,-0.247371,121.82727,11.535055,139.391074
4,Bechar,Algeria,31.61,-2.22,11.083424,-1.245956,29.233835,10.812107,-1.237182,16.658006,-2.425654,2.001824,7.217837


In [30]:
df.to_csv('weather.csv')